In [ ]:
import pandas as pd
import numpy as np

from bokeh.embed import file_html
from bokeh.models import ColumnDataSource, Patches, HoverTool, TapTool, Plot, Range1d
from bokeh.palettes import Blues9
from bokeh.resources import INLINE

from jinja2 import Template
from IPython.display import display_html

from constants import PLOT_FORMATS

### Read in map data and numerical data

In [ ]:
map_data = pd.read_hdf('../cgetools/data/province_map_data.hdf', 'df')
data = pd.read_csv('../cgetools/data/sample_data_by_year.csv')

In [ ]:
print(len(map_data))
map_data.sort('alpha', inplace=True)
map_data.head()

In [ ]:
print(len(data))
data.head(5)

### Join to gether the map data and numerical data

Note if you have a column name matching one in the map_data file (alpha, code, name_en, name_zh), this will work automagically. If not, you can specify columns for the merge.

In [ ]:
all_data = map_data.merge(data)
all_data.head()

#### CHECK THAT YOU HAVE THE EXPECTED NUMBER OF ROWS

In [ ]:
len(all_data) == len(map_data)

# Color your data

In [ ]:
def color_data(data, columns_to_colorify, data_min=None, data_max=None, palette=Blues9):
    # data - the data frame which you are adding colored values to
    # columns_to_colorify - a list of strings which select the columns
    
    if data_min is None:
        num_only = data[columns_to_colorify]
        global_min = num_only.min().min()
        data_min = np.floor(global_min)

    if data_max is None:
        num_only = data[columns_to_colorify]
        global_max = num_only.max().max()
        data_max = np.ceil(global_max)
    
    data_range = data_max - data_min
    bin_factor = data_range / len(palette)
    
    def _get_color(value, palette):
        index = int(value / bin_factor)
        return palette[index - 1]

    for column_name in columns_to_colorify:
        color_name = '%s_color' % column_name
        data[color_name] = data['%s' % column_name].apply(_get_color, args=([palette]))
    return data

colored_data = color_data(all_data, [str(x) for x in range(1990, 2015)])
colored_data[['1990_color']].head()

# Make a static map for 1990 data

In [ ]:
from constants import PLOT_FORMATS

def setup_china_map_plot(column_to_plot, plot_width=600, x_range=[70, 140], y_range=[10, 60], title=""):
    aspect_ratio = (x_range[1] - x_range[0]) / (y_range[1] - y_range[0])
    plot_height = int(plot_width / aspect_ratio)
    x_range = Range1d(x_range[0], x_range[1])
    y_range = Range1d(y_range[0], y_range[1])
    plot = Plot(
        x_range=x_range, 
        y_range=y_range, 
        title=title, 
        plot_width=plot_width, 
        plot_height=plot_height, 
        **PLOT_FORMATS)
    
    countries = Patches(
        xs='xs', 
        ys='ys',
        fill_color=column_to_plot,
        line_color=Blues9[0]
    )    
    
    plot.add_glyph(ColumnDataSource(colored_data), countries)
    return plot

In [ ]:
PLOT_WIDTH = 900
TITLE = '1990 - Random data'
COLUMN_TO_PLOT = '1990_color'

map_box = setup_china_map_plot(COLUMN_TO_PLOT, plot_width=PLOT_WIDTH)

tooltips = "<span class='tooltip-text year'>1990</span>"                   
tooltips += "<span class='tooltip-text country'>@name_en</span>"               
tooltips += "<span class='tooltip-text value'>@1990</span>"                                    
map_box.add_tools(HoverTool(tooltips=tooltips))


# Open our custom template
with open('../cgetools/data/map_template.jinja', 'r') as f:
    template = Template(f.read())

# Update these to change the text
template_variables = {
    'title': TITLE,
    'narrative': 'Some explanatory text.',
}

# Use inline resources, render the html and open
html = file_html(map_box, resources=INLINE, title=TITLE, template=template, template_variables=template_variables)

# Uncomment the next two lines if you'd like to save the file
with open('interactive_map.html', 'w') as f:
    f.write(html)

display_html(html, raw=True)